**1.0 Introdução**

Código com finalidade de realizar a modelagem Cartola FC

**2.0 Pacotes**

In [ ]:
#%pip install catboost
#%pip install shap
#%pip install scikit-learn
#%pip install tensorflow
#%pip install xgboost
#%pip install seaborn

In [ ]:
#tratamento de dados
import pandas as pd
import numpy as np

#ignorar avisos
import warnings
warnings.filterwarnings("ignore")

#modelagem
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

#shap
from tqdm import tqdm
import shap

#grafico
import matplotlib.pyplot as plt
import seaborn as sns

3.0 Dados

In [ ]:
#carregar dados
df = (pd
.read_csv("C:/Users/eduar/OneDrive/Área de Trabalho/Python/cartola_fc/gold/bases/base_final.csv")
)

#remover colunas
df = df.drop(columns=['entrou_em_campo', 'adv','estadio']) 

display(df.head(5))

In [ ]:
rodada_avaliada = 30

#treino
df_train = df.copy()
df_train =  df_train[df_train['rodada'] <= (rodada_avaliada-1)] #rodadas antes

df_train =  df_train[df_train['pontuacao'] <= 22] #outliers
df_train =  df_train[df_train['pontuacao'] <= df_train['jogadores_proAvgpontuacao']*3.5 ] #outliers
df_train =  df_train[df_train['pontuacao'] >= (df_train['jogadores_proAvgpontuacao']-15) ] #outliers


X_train = df_train.drop(columns=['pontuacao','rodada','nome_jogador','equipe','id_jogador']) 
y_train = df_train['pontuacao']

#teste
df_test = df.copy()
df_test =  df_test[df_test['rodada'] == rodada_avaliada]

X_test = df_test.drop(columns=['pontuacao','rodada','nome_jogador','equipe','id_jogador']) 
y_test = df_test['pontuacao']

In [ ]:
#df_train[df_train['rodada'] == 30]

4. Estatística Descritiva

In [ ]:
df['pontuacao'].describe()

In [ ]:
# Calcule o desvio padrão e média para cada jogador
print((df.groupby('id_jogador')['pontuacao'].std()).mean())
print((df.groupby('id_jogador')['pontuacao'].mean()).mean())

In [ ]:
# Crie um gráfico de distribuição usando Seaborn
sns.histplot(df['pontuacao'], kde=True, color='blue')

# Adicione rótulos e título ao gráfico
plt.xlabel('Pontuação')
plt.ylabel('Frequência')
plt.title('Distribuição da Pontuação')

# Exiba o gráfico
plt.show()

5.0 CatBoost

In [ ]:
categorical_var = ['posicao']

#Crie o modelo CatBoost com os parâmetros desejados, incluindo 'data_resposta' como categórica
model = CatBoostRegressor(iterations=200,
                          depth=8,
                          verbose=False,
                          cat_features=categorical_var
                          )

# Treine o modelo no conjunto de treinamento
model.fit(X_train, y_train,
          early_stopping_rounds=5
          )  #Aplica early stopping se o desempenho no conjunto de validação deixar de melhorar

#score de treinamento
print(model.score(X_train, y_train))

# Avalie o modelo no conjunto de teste (ou validação)
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

In [ ]:
X_final = df_test.drop(columns=['pontuacao','rodada','nome_jogador','equipe','id_jogador']) 

# Avalie o modelo no conjunto de teste (ou validação)
y_pred = model.predict(X_final)

#Adicionar uma coluna 'predicao' ao DataFrame com as previsões do modelo para os dados de treinamento
df_test['predicao'] = y_pred.round(2)

#Crie um objeto explainer SHAP
explainer = shap.TreeExplainer(model)

#Calcule os valores SHAP para um conjunto de dados de exemplo
shap_values = explainer.shap_values(X_test)

#Crie um gráfico de resumo SHAP
shap.summary_plot(shap_values, X_test, plot_type="bar") 

In [ ]:
df_test = df_test[['nome_jogador','equipe','posicao','pontuacao','predicao','jogadores_proAvgpontuacao']]
display(df_test.head(5))

In [ ]:
#salvar dado
df_test.to_csv('previsao_geral.csv', index=False,mode='w')

In [ ]:
# import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# # Supondo que 'posicoes' seja a coluna categórica que você deseja transformar
# X_train_dummy = pd.get_dummies(X_train, columns=['posicao'], prefix='posicao', drop_first=True)
# X_test_dummy = pd.get_dummies(X_test, columns=['posicao'], prefix='posicao', drop_first=True)

# # Normalizar os dados usando StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_dummy)
# X_test_scaled = scaler.transform(X_test_dummy)

# # Definir a arquitetura da rede neural
# model_nn = tf.keras.Sequential([
#     tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(X_train_dummy.shape[1],)),
#     tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dense(1)  # Camada de saída para tarefas de regressão
# ])

# # Compilar o modelo
# model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mae'])

# # Treinar o modelo
# model_nn.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_split=0.1, verbose=2)

# # Avaliar o modelo no conjunto de teste
# y_pred_nn = model_nn.predict(X_test_scaled)

# # Avaliar o desempenho do modelo de rede neural
# mse_nn = mean_squared_error(y_test, y_pred_nn)
# r2_nn = r2_score(y_test, y_pred_nn)
# mae_nn = mean_absolute_error(y_test, y_pred_nn)

# print("Mean Absolute Error (MAE) - Rede Neural:", mae_nn)
# print(f"Mean Squared Error (MSE) - Rede Neural: {mse_nn}")
# print(f"R-squared (R2) - Rede Neural: {r2_nn}")

In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# # Supondo que 'posicao' seja a coluna categórica que você deseja transformar
# X_train_dummy = pd.get_dummies(X_train, columns=['posicao'], prefix='posicao', drop_first=True)
# X_test_dummy = pd.get_dummies(X_test, columns=['posicao'], prefix='posicao', drop_first=True)

# # Criar um objeto DMatrix para otimizar o desempenho do XGBoost
# dtrain = xgb.DMatrix(X_train_dummy, label=y_train)
# dtest = xgb.DMatrix(X_test_dummy, label=y_test)

# # Definir os parâmetros do modelo
# params = {
#     'objective': 'reg:squarederror',
#     'eval_metric': 'rmse',
#     'max_depth': 6,
#     'learning_rate': 0.1,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'n_estimators': 100
# }

# # Treinar o modelo
# model_xgb = xgb.train(params, dtrain, num_boost_round=100)

# # Fazer previsões no conjunto de teste
# y_pred_xgb = model_xgb.predict(dtest)

# # Avaliar o desempenho do modelo XGBoost
# mse_xgb = mean_squared_error(y_test, y_pred_xgb)
# r2_xgb = r2_score(y_test, y_pred_xgb)
# mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

# print("Mean Absolute Error (MAE) - XGBoost:", mae_xgb)
# print(f"Mean Squared Error (MSE) - XGBoost: {mse_xgb}")
# print(f"R-squared (R2) - XGBoost: {r2_xgb}")
